# Data Science Interview Prep
## Data Manipulation


1. Let’s say you’re given a list of standardized test scores from high schoolers from grades 9 to 12.

Given the dataset, write code in Pandas to return the cumulative percentage of students that received scores within the buckets of <50, <75, <90, <100.

**My Solution**

In [2]:
import pandas as pd

def getDistribution(grade_df):
    if grade_df.empty:
        # print("Grade chart for grade " + grade_df['grade'].iloc[0] + " is empty")
        return pd.DataFrame({'grade': [], 'test_score': [], 'percentage': []})
    """
    The thresholds we're visualizing
    threshold1 => test scores < 50
    threshold2 => test scores < 75
    threshold3 => test scores < 90
    threshold4 => test scores < 100
    """
    
    #Think of missing values
    threshold1 = len(grade_df[grade_df['test score'] < 50])
    threshold2 = len(grade_df[grade_df['test score'] < 75])
    threshold3 = len(grade_df[grade_df['test score'] < 90])
    threshold4 = len(grade_df[grade_df['test score'] < 100])

    cumulPercent1 = threshold1/len(grade_df['test score']) * 100
    cumulPercent2 = threshold2/len(grade_df['test score']) * 100
    cumulPercent3 = threshold3/len(grade_df['test score']) * 100
    cumulPercent4 = threshold4/len(grade_df['test score']) * 100



    output_grades = pd.DataFrame({'grade': [grade_df['grade'].iloc[0],grade_df['grade'].iloc[0],grade_df['grade'].iloc[0],grade_df['grade'].iloc[0]],'test_score': ['<50', '<75', '<90', '<100'], 'percentage': [cumulPercent1, cumulPercent2, cumulPercent3, cumulPercent4]})
    return output_grades

def getDistributionByGrade(df):
    #Check if dataframe is empty
    if df.empty:
        print("Empty grade chart")

    # Split dataset up into dataframe by grade
    grade9 = df[df['grade'] == 9]
    grade10 = df[df['grade'] == 10]
    grade11 = df[df['grade'] == 11]
    grade12 = df[df['grade'] == 12]

    gradeNineOutput = getDistribution(grade9)
    gradeTenOutput = getDistribution(grade10)
    gradeElevenOutput = getDistribution(grade11)
    gradeTwelveOutput = getDistribution(grade12)

    gradesDistributions = [gradeNineOutput, gradeTenOutput, gradeElevenOutput, gradeTwelveOutput]
    results = pd.concat(gradesDistributions, ignore_index=True)
    
    return results

#Test solution

testFrame = pd.DataFrame({"grade": [10,10,11,10,11], "test score": [85,60,90,30,99]})
output = getDistributionByGrade(testFrame)
print(output)

   grade test_score  percentage
0   10.0        <50   33.333333
1   10.0        <75   66.666667
2   10.0        <90  100.000000
3   10.0       <100  100.000000
4   11.0        <50    0.000000
5   11.0        <75    0.000000
6   11.0        <90    0.000000
7   11.0       <100  100.000000


**More efficient solution**

In [50]:
import numpy as np
df = pd.DataFrame([[1,10,85],[2,10,60],[3,11,90],[4,10,30],[5,11,99]], columns = ["user_id","grade","test score"])

#np.where allows us to grab a numpy array of the test scores, make columns of these counts
df["<50"] = np.where(df["test score"]< 50, 1, 0)
df["<75"] = np.where(df["test score"] < 75, 1, 0)
df["<90"] = np.where(df["test score"] < 90, 1, 0)
df["<100"] = np.where(df["test score"] < 100, 1, 0)

print(df)

   user_id  grade  test score  <50  <75  <90  <100
0        1     10          85    0    0    1     1
1        2     10          60    0    1    1     1
2        3     11          90    0    0    0     1
3        4     10          30    1    1    1     1
4        5     11          99    0    0    0     1


In [51]:
#Now we can group these and get the sum of counts in each bin
df = df.groupby(["grade"])[["<50","<75","<90","<100"]].sum().reset_index()
print(df)

   grade  <50  <75  <90  <100
0     10    1    2    3     3
1     11    0    0    0     2


In [52]:
#This unpivots every column except the grade column using making the test score columns with values from  ["<50","<75","<90","<100"]
#And the count column from the rows associated with the grade
df = df.melt(id_vars=["grade"], var_name="test score", value_name="count")
print(df)

   grade test score  count
0     10        <50      1
1     11        <50      0
2     10        <75      2
3     11        <75      0
4     10        <90      3
5     11        <90      0
6     10       <100      3
7     11       <100      2


In [54]:
#Get the total number of instances in each grade
#We use transform to "paste" the result of our groupby to each row and we use max as the aggregate function because recall that these are running bins that span the range
#Therefore the bin with the highest count, has to contain all the numbers in the subset
df["groupttl"] = df.groupby("grade")["count"].transform('max')

In [55]:
#Get the percentage of each group
df['percentage'] = (df['count'] / df['groupttl']) * 100
print(df)

   grade test score  count  groupttl  percentage
0     10        <50      1         3   33.333333
1     11        <50      0         2    0.000000
2     10        <75      2         3   66.666667
3     11        <75      0         2    0.000000
4     10        <90      3         3  100.000000
5     11        <90      0         2    0.000000
6     10       <100      3         3  100.000000
7     11       <100      2         2  100.000000


In [56]:
#Key in on key columns, sort values
df = (df[['grade', 'test score', 'percentage']].copy().sort_values(['grade', 'percentage'], ascending=True))
print(df)

   grade test score  percentage
0     10        <50   33.333333
2     10        <75   66.666667
4     10        <90  100.000000
6     10       <100  100.000000
1     11        <50    0.000000
3     11        <75    0.000000
5     11        <90    0.000000
7     11       <100  100.000000


In [57]:
#Convert percentage column to string to allow us to cut off trailing decimals and append %
df['percentage'] = df.percentage.astype(int).astype(str)
df['percentage'] = df['percentage'] + '%'
print(df)

   grade test score percentage
0     10        <50        33%
2     10        <75        66%
4     10        <90       100%
6     10       <100       100%
1     11        <50         0%
3     11        <75         0%
5     11        <90         0%
7     11       <100       100%
